# Import

In [2]:
import cv2
import numpy as np

# Initialisation

In [3]:
# First image should be the reference image
srcs = ["template"]
imgs = [] # Liste of images
bws = [] # Liste of image in black and white
for src in srcs:
    img = cv2.imread(src + ".jpg")
    (thresh, bw) = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)
    # print(thresh)
    imgs.append(img)
    bws.append(bw)
    
# print(len(imgs))

# Base image and image in black and white

In [18]:
t = 0
for i in imgs:
    cv2.imshow("image" + str(t), i)
    t += 1
   
t = 0
for bwi in bws:
    cv2.imshow("bw" + str(t), bwi)
    t += 1

cv2.waitKey(0)

-1

# Canny Filter

In [10]:
t = 0
for i in imgs:
    img_canny = cv2.Canny(imgs[t], 200, 200)
    cv2.imshow("canny" + str(t), img_canny)
    t += 1
cv2.waitKey(0)

-1

# Prewitt Filter

In [ ]:
kernelx = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
kernely = np.array([[-1,0,1],[-1,0,1],[-1,0,1]])

In [ ]:
# TODO UPDATE FOR MULTIPLE IMAGE
img_prewittx = cv2.filter2D(bws[0], -1, kernelx)
img_prewitty = cv2.filter2D(bws[0], -1, kernely)

cv2.imshow("prewittx", img_prewittx)
cv2.imshow("prewitty", img_prewitty)
cv2.imshow("prewittx+y", img_prewittx+img_prewitty)
cv2.waitKey(0)

# Sobel Filter

In [ ]:
img_sobelx = cv2.Sobel(bws[0],cv2.CV_8U,1,0,ksize=5)
img_sobely = cv2.Sobel(bws[0],cv2.CV_8U,0,1,ksize=5)
img_sobel = img_sobelx + img_sobely

cv2.imshow("sobel", img_sobel)
cv2.waitKey(0)

# Laplacien Filter

In [ ]:
img_laplacian = cv2.Laplacian(bws[0], cv2.CV_64F)

cv2.imshow("laplacian", img_laplacian)
cv2.waitKey(0)

# New Test

In [17]:
new_img = cv2.imread(srcs[0] + ".jpg")
(new_thresh, new_bw) = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 127, 255, cv2.THRESH_BINARY)

new_bw = cv2.bitwise_not(new_bw)

cv2.imshow("bw", new_bw)
cv2.waitKey(0)

-1

# Morphology

Trying to work with morphology operation for better shape

## Opening

In [18]:
kernel = np.ones((3,3), np.uint8)
opening = cv2.morphologyEx(new_bw, cv2.MORPH_OPEN, kernel, iterations = 2)

## Dilate

In [6]:
sure_bg = cv2.dilate(opening,kernel,iterations=3)


## Distance

In [19]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)

# Show

In [20]:
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)

cv2.imshow("opening", opening)
cv2.imshow("bg", sure_bg)
cv2.imshow("fg", sure_fg)
cv2.imshow("unknown", unknown)


# Shape Matching

## First image

In [ ]:
contoursBase, hierarchyBase = cv2.findContours(sure_bg,
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of Contours found = " + str(len(contours)))
img = cv2.imread(srcs[0] + ".jpg")

cv2.drawContours(img, contours, -1, (0, 255, 0), 3)

cv2.imshow('Contours', img)
cv2.waitKey(0)

Number of Contours found = 1


## Second Image

In [ ]:
contours, hierarchy = cv2.findContours(sure_bg,
                                       cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
print("Number of Contours found = " + str(len(contours)))
img = cv2.imread(srcs[1] + ".jpg")

cv2.drawContours(img, contours, -1, (0, 255, 0), 3)

cv2.imshow('Contours', img)
cv2.waitKey(0)

In [ ]:
for contour in coutnrs:
    res = cv2.matchShapes(contoursBase[0], contour, 1, 0)
    print(res)

# Template Matching